# This notebook show my issue with the calculation time of the enrichment function

In [16]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [17]:
import greatpy as great
import pandas as pd

In [25]:
test = pd.read_csv("../data/human/test_genomic_region.bed",sep="\t",comment="#",
                names=["Chr", "Chr_Start", "Chr_End"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64"})

df_10_row = test[:100]

In [26]:
%lprun -f great.tl.enrichment great.tl.enrichment(df_10_row,"../data/human/regulatory_domain.bed","../data/human/chr_size.bed","../data/human/ontologies.csv",True,True,0.05,(0,0))

Timer unit: 1e-06 s

Total time: 333.221 s
File: /home/tom/.local/lib/python3.8/site-packages/greatpy/tl/basic.py
Function: enrichment at line 279

Line #      Hits         Time  Per Hit   % Time  Line Contents
   279                                           def enrichment(test:str or pd.DataFrame,regdom_file,chr_size_file,annotation,binom=True,hypergeom=True,alpha=0.05,correction=("fdr",0.05),sort_by=None): 
   280                                               # Data import 
   281         1          6.0      6.0      0.0      if not binom and not hypergeom : 
   282                                                   return False
   283                                               
   284         2      36849.0  18424.5      0.0      regdom = pd.read_csv(regdom_file,sep="\t",comment="#",
   285         1          7.0      7.0      0.0                      names=["Chr", "Chr_Start", "Chr_End","Name","tss","Strand"],dtype={"Chr":"object", "Chr_Start":"int64", "Chr_End":"int64","Name":"

Here we can see that the calculation time of the enrichment function is very high due to : 

line 336 (72 %) in the https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with k_binom = number_of_hit(test,curr_regdom) => The number_of_hit function is used to calculate the number of genomic region hit with each GO term 

line 340 (21 %) https://github.com/theislab/greatpy/blob/correction/greatpy/tl/basic.py with dict.update with the values of binom p-value and hypergeom p-value 